In [15]:
import numpy as np
import ase.io as io
from ase import io
import ase.io.lammpsdata as lammps
import os

no_atoms_in_simulation = 511 # number of atoms used in your simulation
masksize = (no_atoms_in_simulation+1)/2 # number of atoms to delete when cutting down cells
path_to_folder = "/home/u2093113/Documents/PaperData/"

#os.mkdir(path_to_folder+"steps")

def cut_cell(atoms,upper,lower,lattice_para,loop_limit,shift_amount):
    incorrect_length = True
    loop = 0
    while incorrect_length:
        mask = []

        # Finds atoms to delete based on x position if statement is to deal with PBC
        if lower > upper:

                for j in range(len(atoms)):
                    if  upper < atoms.get_positions()[j][0] < lower:
                        mask.append(j)
        else : 
                for j in range(len(atoms)):
                    if atoms.get_positions()[j][0] > upper or atoms.get_positions()[j][0] < lower:
                        mask.append(j)
        
        # This section below attempts to allow for when atoms aren't perfectly place and causes the wrong number of atoms to be delete, it does this by shifting the x position of the cut line 
        # a little bit every step to try get the right number of atoms  

        if len(mask)> masksize:
            upper -= shift_amount/loop_limit
            lower -= shift_amount/loop_limit
        if len(mask)< masksize:
            upper += shift_amount/loop_limit
            lower += shift_amount/loop_limit
        if len(mask)== masksize:
            incorrect_length = False
        loop += 1 
        if loop > loop_limit :
            print("This one needs looking at")
            incorrect_length = False


        
    return mask

def cell_cutter(initial,final,j,bad_cells,loop_limit,shift_amount):
    
    atoms = initial.copy()
    nextatoms = final.copy()
    

    lattice_para = atoms.get_cell()[0][0]/8.0
    cell_length = atoms.get_cell()[0][0]


    # Finds atom that is moving the most
    diff = atoms.get_positions()-nextatoms.get_positions()
    store = 0
    for i in diff:
        current = np.linalg.norm(i)
        if current > store:
            store = current
            entry = i
    

    diffcheck =  nextatoms.get_positions()[np.where(entry==diff)[0][0]] - atoms.get_positions()[np.where(entry==diff)[0][0]]
    
    center_pos = atoms.get_positions()[np.where(entry==diff)[0][0]]

    center_pos = center_pos + 0.25*np.array([lattice_para,lattice_para,lattice_para])


    # Moves cell to centre on moving atom then wraps around PBC
    atoms.center(about=-center_pos)
    atoms.wrap()
    atoms.center()
    
    nextatoms.center(about=-center_pos)
    nextatoms.wrap()
    nextatoms.center()
    
    if diffcheck[0] < 0:
        lower_shift = 2.5
        upper_shift = 1.5
    else: 
        lower_shift = 1.5
        upper_shift = 2.5

    center_pos = atoms.get_positions()[np.where(entry==diff)[0][0]]

    center_pos = center_pos + 0.25*np.array([lattice_para,lattice_para,lattice_para])
    upper = (center_pos[0] + upper_shift*lattice_para)%cell_length
    lower = (center_pos[0] - lower_shift*lattice_para)%cell_length
    
    mask = cut_cell(atoms,upper,lower,lattice_para,loop_limit,shift_amount)
    del atoms[mask]
    atoms.set_cell([4.0*lattice_para,4.0*lattice_para,4.0*lattice_para])
    atoms.center()
    del nextatoms[mask]
    nextatoms.set_cell([4.0*lattice_para,4.0*lattice_para,4.0*lattice_para])
    nextatoms.center()
    

    # Attemps to catalogue what goes wrong and when
    if len(nextatoms) != 255:
        print("oh no")
        
    if len(mask) != 256:  
        bad_cells.append(j)
    
    ase_init=path_to_folder+"steps/cutDownCellStep{}initial.xyz".format(j) #
    ase_final=path_to_folder+"steps/cutDownCellStep{}final.xyz".format(j)
    lammps_init=path_to_folder+"steps/cutDownCellStep{}initial.txt".format(j)
    lammps_final= path_to_folder+"steps/cutDownCellStep{}final.txt".format(j)
    
    io.write(ase_init,atoms)
    io.write(ase_final,nextatoms)
    lammps.write_lammps_data(lammps_init,atoms)
    lammps.write_lammps_data(lammps_final,nextatoms)

In [16]:
#writes out configurations at each step of KMC
statesList = io.read(path_to_folder+"allconf_defect",index=":")
statesList = statesList[::2]
NoState = len(statesList)
for i in range(NoState):
    io.write(path_to_folder+"step{}.xyz".format(i),statesList[i])
    lammps.write_lammps_data(path_to_folder+"step{}.txt".format(i),statesList[i])



In [17]:
bad_cells = [] #highlights any cells that have the wrong number of atoms 

for j in range(NoState-1):
    
    cell_cutter(statesList[j],statesList[j+1],j,bad_cells,1000,0)  # Last 2 numbers one to be changed if there is difficulty in cutting down cells 